In [1]:
import findspark
from datetime import datetime as dt
findspark.init()

In [8]:
from pyspark.sql import SparkSession
from os import path
spark = SparkSession.builder.appName('Test').getOrCreate()
from pyspark.sql.functions import lit ,col
from pyspark.sql.functions import current_date

In [4]:
df=spark.read.csv('refined/data1.csv', header = True)
df.show()
# df2 = df.withColumn("update_date", lit(str(dt.now()))).withColumn("active_flag", lit(True)  )

# #df2.write.format("csv").save('hco/final')
# df2.show()

#def uspertfunc(df):
    

+-----+----+----+
|hcoid|name|type|
+-----+----+----+
|    1|   a|  qw|
|    2|   b|  er|
|    3|   c|  ty|
|    4|   d|  ui|
|    5|   e|  op|
|    6|   f|  as|
|    7|   g|  df|
|    8|   h|  gh|
|    9|   i|  jk|
|   10|   j|  zx|
+-----+----+----+



In [77]:
wr_path = 'hco/final'
col_names = ['hcoid','name','type', 'update_date','active_flag']

if path.exists("hco/final"):
    df=spark.read.csv(wr_path, header = True)
    print('existing DF')
    df.show()
    ddf=spark.read.csv('refined/data2.csv', header = True)
    ddf = ddf.withColumn("update_date", lit(str(dt.now()))).withColumn("active_flag", lit('Y')  )
    ddf_f = ddf
    print('delta DF')
    ddf_f.show()
    df = df.withColumn("hcoid_m",df["hcoid"])
    ddf = ddf.withColumn("hcoid_d",ddf["hcoid"])
    #Joining Existing DF with Delta DF
    joineddf= df.join(ddf,df.hcoid_m ==  ddf.hcoid_d,"inner")
    delta_ids = joineddf.select("hcoid_m").collect()
    #convert student Name to list using
    # flatMap
    lst = joineddf.select('hcoid_m').rdd.flatMap(lambda x: x).collect()
    print('Delta IDs',lst)
    print("Existing DF minus updates")
    df_sub_delta = df.where(~col("hcoid").isin(lst))
    df_sub_delta = df_sub_delta.select(col_names)
    df_sub_delta.show()
    print("Existing DF only updates with active flag N")
    df_only_delta = df.where(col("hcoid").isin(lst))
    df_only_delta = df_only_delta.select(col_names[:3])
    df_only_delta = df_only_delta.withColumn("update_date", lit(str(dt.now()))).withColumn("active_flag", lit('N'))
    df_only_delta.show()
    final = ddf_f.union(df_sub_delta).union(df_only_delta)
    print('Final Data')
    final.show()
    final.repartition(1).write.format("csv").mode("overwrite").option("header","true").save('hco_temp/final')
    
else:
    print('No files in the path',e)
    df2=spark.read.csv('refined/data1.csv', header = True)
    df2 = df2.withColumn("update_date", lit(str(dt.now()))).withColumn("active_flag", lit('Y')  )
    df2.show()
    df2.write.format("csv").mode("overwrite").option("header","true").save('hco/final')  
    


existing DF
+-----+----+----+--------------------+-----------+
|hcoid|name|type|         update_date|active_flag|
+-----+----+----+--------------------+-----------+
|    1|   a|  qw|2022-03-05 17:21:...|          Y|
|    2|   b|  er|2022-03-05 17:21:...|          Y|
|    3|   c|  ty|2022-03-05 17:21:...|          Y|
|    4|   d|  ui|2022-03-05 17:21:...|          Y|
|    5|   e|  op|2022-03-05 17:21:...|          Y|
|    6|   f|  as|2022-03-05 17:21:...|          Y|
|    7|   g|  df|2022-03-05 17:21:...|          Y|
|    8|   h|  gh|2022-03-05 17:21:...|          Y|
|    9|   i|  jk|2022-03-05 17:21:...|          Y|
|   10|   j|  zx|2022-03-05 17:21:...|          Y|
+-----+----+----+--------------------+-----------+

delta DF
+-----+----+----+--------------------+-----------+
|hcoid|name|type|         update_date|active_flag|
+-----+----+----+--------------------+-----------+
|    2|   b|  ab|2022-03-05 18:41:...|          Y|
|    3|   c|  ty|2022-03-05 18:41:...|          Y|
|    7|  

In [71]:
print(dt.now())

2022-02-28 01:10:01.395145


In [106]:
ddf=spark.read.csv('refined/data2.csv', header = True)
ddf = ddf.withColumn("update_date", lit(str(dt.now())))
ddf.show()

+-----+----+----+--------------------+
|hcoid|name|type|         update_date|
+-----+----+----+--------------------+
|    2|   b|  ab|2022-02-28 01:31:...|
|    3|   c|  ty|2022-02-28 01:31:...|
|    7|   g|  df|2022-02-28 01:31:...|
|   10|   j|  zx|2022-02-28 01:31:...|
|   11|   k|  zq|2022-02-28 01:31:...|
|   12|   l|  xw|2022-02-28 01:31:...|
+-----+----+----+--------------------+



In [112]:
df2=spark.read.csv('refined/data1.csv', header = True)
df2 = df2.withColumn("update_date", lit(str(dt.now()))).withColumn("active_flag",  lit(str('Y')) )
df2.show()
#df.show()
#df2.write.format("csv").mode("overwrite").option("header","true").save('hco/final')

+-----+----+----+--------------------+-----------+
|hcoid|name|type|         update_date|active_flag|
+-----+----+----+--------------------+-----------+
|    1|   a|  qw|2022-02-28 01:32:...|          Y|
|    2|   b|  er|2022-02-28 01:32:...|          Y|
|    3|   c|  ty|2022-02-28 01:32:...|          Y|
|    4|   d|  ui|2022-02-28 01:32:...|          Y|
|    5|   e|  op|2022-02-28 01:32:...|          Y|
|    6|   f|  as|2022-02-28 01:32:...|          Y|
|    7|   g|  df|2022-02-28 01:32:...|          Y|
|    8|   h|  gh|2022-02-28 01:32:...|          Y|
|    9|   i|  jk|2022-02-28 01:32:...|          Y|
|   10|   j|  zx|2022-02-28 01:32:...|          Y|
+-----+----+----+--------------------+-----------+

